# Layer Activation Function Comparison

In standard VGG-Net architectures, ReLU activation is used. This 
notebook will explore whether other functions would give our model
better performance.

We shall look at the ReLU, Sigmoid, and Tanh functions. For simplicity,
we shall constrain all convolutional layers to share the same activation 
function, and the same goes for the hidden dense layers. 

In [ ]:
from gc import collect
from typing import Tuple

import matplotlib.pyplot as plt
import numpy as np
from h5py import File
from pandas import read_csv
from sklearn.metrics import accuracy_score, classification_report, \
    confusion_matrix
from tensorflow.config.experimental import list_physical_devices, \
    set_memory_growth
from tensorflow.keras.backend import clear_session
from tensorflow.keras.callbacks import CSVLogger, LearningRateScheduler, \
    ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Input, MaxPool2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.random import set_random_seed


### Model Creation Function

In [ ]:
def create_model(input_shape: Tuple[int, int, int], num_classes: int, 
                 conv_activation: str = "relu", 
                 dense_activation: str = "relu") -> Model:
    inputs = Input(shape=input_shape)
    
    layer = Conv2D(filters=64, kernel_size=(3, 3), strides=1, padding="same", 
                   activation=conv_activation, kernel_initializer="he_normal",
                   bias_initializer="he_normal")(inputs)
    layer = Conv2D(filters=64, kernel_size=(3, 3), strides=1, padding="same", 
                   activation=conv_activation, kernel_initializer="he_normal",
                   bias_initializer="he_normal")(layer)
    layer = MaxPool2D(pool_size=(2, 2), strides=2)(layer)

    layer = Conv2D(filters=128, kernel_size=(3, 3), strides=1, padding="same", 
                   activation=conv_activation, kernel_initializer="he_normal",
                   bias_initializer="he_normal")(layer)
    layer = Conv2D(filters=128, kernel_size=(3, 3), strides=1, padding="same", 
                   activation=conv_activation, kernel_initializer="he_normal",
                   bias_initializer="he_normal")(layer)
    layer = MaxPool2D(pool_size=(2, 2), strides=2)(layer)
    
    layer = Conv2D(filters=256, kernel_size=(3, 3), strides=1, padding="same", 
                   activation=conv_activation, kernel_initializer="he_normal",
                   bias_initializer="he_normal")(layer)
    layer = Conv2D(filters=256, kernel_size=(3, 3), strides=1, padding="same", 
                   activation=conv_activation, kernel_initializer="he_normal",
                   bias_initializer="he_normal")(layer)
    layer = MaxPool2D(pool_size=(2, 2), strides=2)(layer)
    
    layer = Conv2D(filters=512, kernel_size=(3, 3), strides=1, padding="same", 
                   activation=conv_activation, kernel_initializer="he_normal",
                   bias_initializer="he_normal")(layer)
    layer = Conv2D(filters=512, kernel_size=(3, 3), strides=1, padding="same", 
                   activation=conv_activation, kernel_initializer="he_normal",
                   bias_initializer="he_normal")(layer)
    layer = MaxPool2D(pool_size=(2, 2), strides=2)(layer)
    
    layer = Conv2D(filters=512, kernel_size=(3, 3), strides=1, padding="same", 
                   activation=conv_activation, kernel_initializer="he_normal",
                   bias_initializer="he_normal")(layer)
    layer = Conv2D(filters=512, kernel_size=(3, 3), strides=1, padding="same", 
                   activation=conv_activation, kernel_initializer="he_normal",
                   bias_initializer="he_normal")(layer)
    layer = MaxPool2D(pool_size=(2, 2), strides=2)(layer)
    
    layer = Flatten()(layer)
    layer = Dense(units=4096, activation=dense_activation, 
                  kernel_initializer="he_normal",
                  bias_initializer="he_normal")(layer)
    layer = Dense(units=4096, activation=dense_activation, 
                  kernel_initializer="he_normal",
                  bias_initializer="he_normal")(layer)
    layer = Dense(num_classes, activation="softmax")(layer)
    
    model = Model(inputs=inputs, outputs=layer)
    model.compile(optimizer=Adam(learning_rate=0.0001), 
                  loss="categorical_crossentropy", metrics=["accuracy"])
    
    return model


### Other Functions


In [ ]:
def refresh_session():
    # Call this before training a new model, to free up memory from the 
    # previous model
    clear_session()
    try:
        del model
    except NameError:
        pass
    collect()
    
    
def import_dataset(filepath: str = "./dataset.hdf5") \
        -> Tuple[np.ndarray, np.ndarray, np.ndarray, 
                 np.ndarray, np.ndarray, np.ndarray]:
    file = File(filepath, "r")
    train_data = file.get("tr_data")[()]
    val_data = file.get("val_data")[()]
    test_data = file.get("ts_data")[()]
    train_labels = file.get("tr_labels")[()]
    val_labels = file.get("val_labels")[()]
    test_labels = file.get("ts_labels")[()]
    
    return train_data, val_data, test_data, \
           train_labels, val_labels, test_labels


def get_test_results(test_model: Model, test_data: np.ndarray, 
                     test_labels: np.ndarray) -> Tuple:
    predicts = test_model.predict(test_data)
    pred_out = np.argmax(predicts, axis=1)
    test_out = np.argmax(test_labels, axis=1)
    labels = ["car", "heavy vehicles", "motorcycle"]
    
    return accuracy_score(test_out, pred_out), \
           confusion_matrix(test_out, pred_out), \
           classification_report(test_out, pred_out, target_names=labels)


def get_learn_rate(epoch: int) -> float:
    if epoch <= 10:
        lr = 1e-4
    elif epoch <= 20:
        lr = 5e-5
    elif epoch <= 30:
        lr = 1e-5
    elif epoch <= 40:
        lr = 5e-6
    else:
        lr = 1e-6
    print(f"Learning rate: {lr}")
    
    return lr


### Initialise Environment

In [ ]:
# Configure tensorflow to optimise GPU utilisation
gpu_list = list_physical_devices("GPU")
for gpu in gpu_list:
    set_memory_growth(gpu, True)
del gpu_list

# Fix tensorflow random seed
set_random_seed(324)

tr_dat, val_dat, ts_dat, tr_lbls, val_lbls, ts_lbls = import_dataset()

in_shape = (tr_dat.shape[1], tr_dat.shape[2], tr_dat.shape[3])
num_cls = tr_lbls.shape[1]

lr_scheduler = LearningRateScheduler(get_learn_rate)
early_stopper = EarlyStopping(monitor="val_loss", patience=10, verbose=1, 
                              mode="min", restore_best_weights=True) 

# Test data is not needed in this notebook, so free up the memory
del ts_dat
del ts_lbls
collect()


### Model Creation and Training
 

In [ ]:
# tanh conv activation
refresh_session()
data_gen = ImageDataGenerator(
    rotation_range=45, width_shift_range=0.2, height_shift_range=0.2,
    zoom_range=0.2, horizontal_flip=True)

conv_activation = "tanh"
model = create_model(in_shape, num_cls, conv_activation=conv_activation)
print(model.summary())

checkpoint = ModelCheckpoint("./trained_models/tanh_conv_activation_best.hdf5", 
                             monitor="val_loss", verbose=0, 
                             save_best_only=True, mode="min")
logger = CSVLogger("./training_logs/tanh_conv_activation_log.csv")

model.fit_generator(
    data_gen.flow(tr_dat, tr_lbls, batch_size=32, shuffle=True), 
    steps_per_epoch=(len(tr_dat) / 32), epochs=50, verbose=2, 
    callbacks=[checkpoint, logger, lr_scheduler, early_stopper], 
    validation_data=(val_dat, val_lbls))


In [ ]:
# sigmoid conv activation
refresh_session()
data_gen = ImageDataGenerator(
    rotation_range=45, width_shift_range=0.2, height_shift_range=0.2,
    zoom_range=0.2, horizontal_flip=True)

conv_activation = "sigmoid"
model = create_model(in_shape, num_cls, conv_activation=conv_activation)
print(model.summary())

checkpoint = ModelCheckpoint("./trained_models/sigmoid_conv_activation_best.hdf5", 
                             monitor="val_loss", verbose=0, 
                             save_best_only=True, mode="min")
logger = CSVLogger("./training_logs/sigmoid_conv_activation_log.csv")

model.fit_generator(
    data_gen.flow(tr_dat, tr_lbls, batch_size=32, shuffle=True), 
    steps_per_epoch=(len(tr_dat) / 32), epochs=50, verbose=2, 
    callbacks=[checkpoint, logger, lr_scheduler, early_stopper], 
    validation_data=(val_dat, val_lbls))


In [ ]:
# tanh dense activation
refresh_session()
data_gen = ImageDataGenerator(
    rotation_range=45, width_shift_range=0.2, height_shift_range=0.2,
    zoom_range=0.2, horizontal_flip=True)

dense_activation = "tanh"
model = create_model(in_shape, num_cls, dense_activation=dense_activation)
print(model.summary())

checkpoint = ModelCheckpoint(
    "./trained_models/tanh_dense_activation_best.hdf5", monitor="val_loss", 
    verbose=0, save_best_only=True, mode="min")
logger = CSVLogger("./training_logs/tanh_dense_activation_log.csv")

model.fit_generator(
    data_gen.flow(tr_dat, tr_lbls, batch_size=32, shuffle=True), 
    steps_per_epoch=(len(tr_dat) / 32), epochs=50, verbose=2, 
    callbacks=[checkpoint, logger, lr_scheduler, early_stopper], 
    validation_data=(val_dat, val_lbls))


In [ ]:
# sigmoid dense activation
refresh_session()
data_gen = ImageDataGenerator(
    rotation_range=45, width_shift_range=0.2, height_shift_range=0.2,
    zoom_range=0.2, horizontal_flip=True)

dense_activation = "sigmoid"
model = create_model(in_shape, num_cls, dense_activation=dense_activation)
print(model.summary())

checkpoint = ModelCheckpoint(
    "./trained_models/sigmoid_dense_activation_best.hdf5", monitor="val_loss", 
    verbose=0, save_best_only=True, mode="min")
logger = CSVLogger("./training_logs/sigmoid_dense_activation_log.csv")

model.fit_generator(
    data_gen.flow(tr_dat, tr_lbls, batch_size=32, shuffle=True), 
    steps_per_epoch=(len(tr_dat) / 32), epochs=50, verbose=2, 
    callbacks=[checkpoint, logger, lr_scheduler, early_stopper], 
    validation_data=(val_dat, val_lbls))


In [ ]:
# Training phase is complete: free training data memory
del tr_dat
del tr_lbls
refresh_session()


### Data Visualisation

In [ ]:
acc_scores = dict()
conf_matrices = dict()
class_reports = dict()

plt.style.use("ggplot")


In [ ]:
# relu baseline
refresh_session()
model = load_model("./trained_models/vgg13_best.hdf5")
acc_scores["relu"], conf_matrices["relu"], \
class_reports["relu"] = get_test_results(model, val_dat, val_lbls)

print("Validation accuracy for baseline relu activation: "
      f"{acc_scores['relu']}")
print("Confusion Matrix:")
print(conf_matrices["relu"])
print(class_reports["relu"])

log_relu = read_csv("./training_logs/vgg13_log.csv")

plt.figure(figsize=[12.5, 12.5])
plt.subplot(311)
plt.plot(log_relu["val_loss"], label="loss")
plt.plot(log_relu["val_acc"], label="accuracy")
plt.title("Validation")
plt.legend(loc="upper left", bbox_to_anchor=(1.0, 1.0))

plt.subplot(312)
plt.plot(log_relu["loss"], label="train loss")
plt.plot(log_relu["val_loss"], label="validation loss")
plt.title("Loss")
plt.legend(loc="upper left", bbox_to_anchor=(1.0, 1.0))

plt.subplot(313)
plt.plot(log_relu["acc"], label="train accuracy")
plt.plot(log_relu["val_acc"], label="validation accuracy")
plt.title("Accuracy")
plt.legend(loc="upper left", bbox_to_anchor=(1.0, 1.0))


In [ ]:
# tanh conv activation
refresh_session()
model = load_model("./trained_models/tanh_conv_activation_best.hdf5")
acc_scores["tanh_conv"], conf_matrices["tanh_conv"], \
class_reports["tanh_conv"] = get_test_results(model, val_dat, val_lbls)

print("Validation accuracy for tanh conv activation: "
      f"{acc_scores['tanh_conv']}")
print("Confusion Matrix:")
print(conf_matrices["tanh_conv"])
print(class_reports["tanh_conv"])

log_tanh_conv = read_csv("./training_logs/tanh_conv_activation_log.csv")

plt.figure(figsize=[12.5, 12.5])
plt.subplot(311)
plt.plot(log_tanh_conv["val_loss"], label="loss")
plt.plot(log_tanh_conv["val_acc"], label="accuracy")
plt.title("Validation")
plt.legend(loc="upper left", bbox_to_anchor=(1.0, 1.0))

plt.subplot(312)
plt.plot(log_tanh_conv["loss"], label="train loss")
plt.plot(log_tanh_conv["val_loss"], label="validation loss")
plt.title("Loss")
plt.legend(loc="upper left", bbox_to_anchor=(1.0, 1.0))

plt.subplot(313)
plt.plot(log_tanh_conv["acc"], label="train accuracy")
plt.plot(log_tanh_conv["val_acc"], label="validation accuracy")
plt.title("Accuracy")
plt.legend(loc="upper left", bbox_to_anchor=(1.0, 1.0))


In [ ]:
# sigmoid conv activation
refresh_session()
model = load_model("./trained_models/sigmoid_conv_activation_best.hdf5")
acc_scores["sigmoid_conv"], conf_matrices["sigmoid_conv"], \
class_reports["sigmoid_conv"] = get_test_results(model, val_dat, val_lbls)

print("Validation accuracy for sigmoid conv activation: "
      f"{acc_scores['sigmoid_conv']}")
print("Confusion Matrix:")
print(conf_matrices["sigmoid_conv"])
print(class_reports["sigmoid_conv"])

log_sigmoid_conv = read_csv("./training_logs/sigmoid_conv_activation_log.csv")

plt.figure(figsize=[12.5, 12.5])
plt.subplot(311)
plt.plot(log_sigmoid_conv["val_loss"], label="loss")
plt.plot(log_sigmoid_conv["val_acc"], label="accuracy")
plt.title("Validation")
plt.legend(loc="upper left", bbox_to_anchor=(1.0, 1.0))

plt.subplot(312)
plt.plot(log_sigmoid_conv["loss"], label="train loss")
plt.plot(log_sigmoid_conv["val_loss"], label="validation loss")
plt.title("Loss")
plt.legend(loc="upper left", bbox_to_anchor=(1.0, 1.0))

plt.subplot(313)
plt.plot(log_sigmoid_conv["acc"], label="train accuracy")
plt.plot(log_sigmoid_conv["val_acc"], label="validation accuracy")
plt.title("Accuracy")
plt.legend(loc="upper left", bbox_to_anchor=(1.0, 1.0))


In [ ]:
# tanh dense activation
refresh_session()
model = load_model("./trained_models/tanh_dense_activation_best.hdf5")
acc_scores["tanh_dense"], conf_matrices["tanh_dense"], \
class_reports["tanh_dense"] = get_test_results(model, val_dat, val_lbls)

print("Validation accuracy for tanh dense activation: "
      f"{acc_scores['tanh_dense']}")
print("Confusion Matrix:")
print(conf_matrices["tanh_dense"])
print(class_reports["tanh_dense"])

log_tanh_dense = read_csv("./training_logs/tanh_dense_activation_log.csv")

plt.figure(figsize=[12.5, 12.5])
plt.subplot(311)
plt.plot(log_tanh_dense["val_loss"], label="loss")
plt.plot(log_tanh_dense["val_acc"], label="accuracy")
plt.title("Validation")
plt.legend(loc="upper left", bbox_to_anchor=(1.0, 1.0))

plt.subplot(312)
plt.plot(log_tanh_dense["loss"], label="train loss")
plt.plot(log_tanh_dense["val_loss"], label="validation loss")
plt.title("Loss")
plt.legend(loc="upper left", bbox_to_anchor=(1.0, 1.0))

plt.subplot(313)
plt.plot(log_tanh_dense["acc"], label="train accuracy")
plt.plot(log_tanh_dense["val_acc"], label="validation accuracy")
plt.title("Accuracy")
plt.legend(loc="upper left", bbox_to_anchor=(1.0, 1.0))


In [ ]:
# sigmoid dense activation
refresh_session()
model = load_model("./trained_models/sigmoid_dense_activation_best.hdf5")
acc_scores["sigmoid_dense"], conf_matrices["sigmoid_dense"], \
class_reports["sigmoid_dense"] = get_test_results(model, val_dat, val_lbls)

print("Validation accuracy for sigmoid dense activation: "
      f"{acc_scores['sigmoid_dense']}")
print("Confusion Matrix:")
print(conf_matrices["sigmoid_dense"])
print(class_reports["sigmoid_dense"])

log_sigmoid_dense = read_csv("./training_logs/sigmoid_dense_activation_log.csv")

plt.figure(figsize=[12.5, 12.5])
plt.subplot(311)
plt.plot(log_sigmoid_dense["val_loss"], label="loss")
plt.plot(log_sigmoid_dense["val_acc"], label="accuracy")
plt.title("Validation")
plt.legend(loc="upper left", bbox_to_anchor=(1.0, 1.0))

plt.subplot(312)
plt.plot(log_sigmoid_dense["loss"], label="train loss")
plt.plot(log_sigmoid_dense["val_loss"], label="validation loss")
plt.title("Loss")
plt.legend(loc="upper left", bbox_to_anchor=(1.0, 1.0))

plt.subplot(313)
plt.plot(log_sigmoid_dense["acc"], label="train accuracy")
plt.plot(log_sigmoid_dense["val_acc"], label="validation accuracy")
plt.title("Accuracy")
plt.legend(loc="upper left", bbox_to_anchor=(1.0, 1.0))


#### Cross-Model Comparison

In [ ]:
plt.figure(figsize=[10, 7.5])
plt.subplot(211)
plt.plot(log_relu["val_loss"], label="relu baseline")
plt.plot(log_sigmoid_conv["val_loss"], label="sigmoid conv")
plt.plot(log_tanh_conv["val_loss"], label="tanh conv")
plt.title("Validation Loss")
plt.legend(loc="upper left", bbox_to_anchor=(1.0, 1.0))

plt.subplot(212)
plt.plot(log_relu["val_acc"], label="relu baseline")
plt.plot(log_sigmoid_conv["val_acc"], label="sigmoid conv")
plt.plot(log_tanh_conv["val_acc"], label="tanh conv")
plt.title("Validation Accuracy")
plt.legend(loc="upper left", bbox_to_anchor=(1.0, 1.0))

In [ ]:
plt.figure(figsize=[10, 7.5])
plt.subplot(211)
plt.plot(log_relu["val_loss"], label="relu baseline")
plt.plot(log_sigmoid_dense["val_loss"], label="sigmoid dense")
plt.plot(log_tanh_dense["val_loss"], label="tanh dense")
plt.title("Validation Loss")
plt.legend(loc="upper left", bbox_to_anchor=(1.0, 1.0))

plt.subplot(212)
plt.plot(log_relu["val_acc"], label="relu baseline")
plt.plot(log_sigmoid_dense["val_acc"], label="sigmoid dense")
plt.plot(log_tanh_dense["val_acc"], label="tanh dense")
plt.title("Validation Accuracy")
plt.legend(loc="upper left", bbox_to_anchor=(1.0, 1.0))



